# Generating DG Data using Simbench and Powerdata-gen

[1] https://github.com/e2nIEE/simbench  
[2] https://github.com/bdonon/powerdata-gen

### Load dependencies, including the data generator library

In [1]:
import sys, os
DATA_GEN_PATH = os.path.abspath('powerdata-gen/')
sys.path.append(DATA_GEN_PATH)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
import pandas as pd
import pandapower as pp
import simbench as sb
import powerdata_gen

import time
import logging

### Helper functions to load pandapower grids from simbench

In [2]:
def create_output_dir():
    identifier = time.strftime('%Y-%m-%d_%H:%M:%S')
    output_dir = os.path.join('outputs', identifier)
    os.makedirs(output_dir, exist_ok=True)
    return output_dir

In [3]:
def get_dist_grid_codes():
    # Create the codes for the distribution grid cases of Simbench (LV and MV and any combination of the two)
    MV_as_l_voltage_level = sb.collect_all_simbench_codes(lv_level="MV")
    MV_as_h_voltage_level = sb.collect_all_simbench_codes(hv_level="HV")
    LV_as_l_voltage_level = sb.collect_all_simbench_codes(lv_level="LV")
    LV_as_h_voltage_level = sb.collect_all_simbench_codes(hv_level="LV")

    dist_grid_codes = set(MV_as_l_voltage_level + MV_as_h_voltage_level + LV_as_l_voltage_level + LV_as_h_voltage_level)
    dist_grid_codes = list(filter(lambda x: "HV" not in x and "complete" not in x, dist_grid_codes))
    return dist_grid_codes

In [4]:
def save_pandapower_grid_to_json(sb_code: str, filename: str):
    net = sb.get_simbench_net(sb_code)
    pp.to_json(net, filename)
    return filename

### Load a base config file and change desired parameters for each synthetic base grid

In [5]:
from omegaconf import OmegaConf

cfg = OmegaConf.load('base_gen_config.yaml')
cfg.n_train = 1
cfg.n_val = 1
cfg.n_test = 1
cfg.seed = 12

### Generate Grids using powerdata-gen

In [6]:
TEST_GENERATION = True
# TEST_SB_CODES = ['1-MVLV-urban-all-1-no_sw']
TEST_SB_CODES = ['1-LV-rural1--0-sw']

# Setup input directory
input_dir = 'inputs/'
os.makedirs(input_dir, exist_ok=True)

# Get simbench codes and save them to json files, if not already done.
sb_codes = get_dist_grid_codes()
sb_codes = TEST_SB_CODES if TEST_GENERATION else sb_codes
filenames = []
for code in sb_codes:
    f = os.path.join(input_dir, f'{code}.json')
    if not os.path.exists(f):
        save_pandapower_grid_to_json(code, f)
    filenames.append(f)

# Set up logger (for powerdata-gen)
log = logging.getLogger(__name__)

# Create output directory for all data from this run, loop through ref grids, 
# generate new grids for each ref, save them to subdir of output dir.
output_dir = create_output_dir()
generated_grid_base_dirs = []
for code, f in list(zip(sb_codes, filenames)):
    save_path = os.path.join(output_dir, code)
    os.makedirs(save_path, exist_ok=True)
    cfg.default_net_path = f
    powerdata_gen.build_datasets(cfg.default_net_path, save_path, log, cfg.n_train, cfg.n_val, cfg.n_test, cfg.keep_reject,
                   cfg.sampling, cfg.powerflow, cfg.filtering, cfg.seed)
    generated_grid_base_dirs.append(save_path)

Sample count = 1, Sampling issues = 0, Divergences = 0, Rejections = 0 : 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]
Sample count = 1, Sampling issues = 0, Divergences = 0, Rejections = 0 : 100%|██████████| 1/1 [00:00<00:00,  6.28it/s]
Sample count = 4, Sampling issues = 0, Divergences = 0, Rejections = 3 : 100%|██████████| 1/1 [00:00<00:00,  4.19it/s]


### Helper functions for extracting node features and edge features

In [8]:
def get_node_features(net):
    # List of bus features
    #   x: np.array([Slack?, PV?, PQ?, p_mw, q_mvar, vm_pu, va_degree])
    #   y: np.array([p_mw, q_mvar, vm_pu, va_degree])
    #
    node_features_x, node_features_y = [], [] # map from bus_id to features
    for bus_id in net.bus.index:
        # (Slack?, PV?, PQ?)
        bus_type = (0, 0, 1)

        gens = net.gen.loc[net.gen['bus'] == bus_id]
        if len(gens) > 0:
            bus_type = (0, 1, 0)

        slack = net.ext_grid.loc[net.ext_grid['bus'] == bus_id,
                        ['vm_pu', 'va_degree']]
        if len(slack) > 0:
            assert len(gens) == 0, ("PV and Swing generators cannot be placed"
                                    " on the same bus. This is because they"
                                    " will both try to control the bus voltage.")
            bus_type = (1, 0, 0)
        
        # net.res_bus should already take into account all the components that
        # contribute to these four bus parameters so we do not have to do this
        # again (ex. loads, sgens, gens, storages, ext_grid, etc.).
        features = net.res_bus.loc[bus_id, ['p_mw', 'q_mvar', 'vm_pu', 'va_degree']]
        masked_features = features.copy()
        if bus_type[0]:
            masked_features['p_mw'] = np.nan
            masked_features['q_mvar'] = np.nan
        elif bus_type[1]:
            masked_features['q_mvar'] = np.nan
            masked_features['va_degree'] = np.nan
        else:
            masked_features['vm_pu'] = np.nan
            masked_features['va_degree'] = np.nan

        node_features_x.append(np.append(bus_type, masked_features.values))
        node_features_y.append(features.values)
    
    return np.array(node_features_x), np.array(node_features_y)

In [9]:
def get_edge_features(net):
    # List of edge features
    #   e: np.array([trafo?, r_pu, x_pu, phase_shift])

    def get_line_features(net):
        # Undirected graph so need to add both directions to edge_index.
        edge_index = net.line.loc[:, ['from_bus', 'to_bus',
                                      'to_bus', 'from_bus']].values
        # Use .reshape to change shape from (E, 4) to (2E, 2), where E is num edges.
        # Transpose to make into proper (2, 2E format).
        edge_index = edge_index.reshape(-1, 2).T

        # TODO: Decide if use r/x or G/B??
        r = net.line['r_ohm_per_km'].values * net.line['length_km'].values
        x = net.line['x_ohm_per_km'].values * net.line['length_km'].values

        # We convert the r,x values into per unit (p.u.) to simplify calculations
        # and ensure consistency across the network. To do this, we divide r, x by
        # the base impedance. Therefore z = vn_kv**2/sn_mva, where vn_kv is rated
        # voltage and sn_mva is reference apparent power.
        # Note: vn_kv be the same for every bus except ext_grid, but this is safer.
        vn_kv = net.bus.loc[net.line['to_bus'], ['vn_kv']].values.reshape(-1)
        z = np.square(vn_kv) / net.sn_mva
        r_pu = r / z
        x_pu = x / z

        # Similarly, due to undirected graph, the edge features need to be repeated
        # twice, once for each respective connection present in the COO matrix.
        r_pu = r_pu.repeat(2)
        x_pu = x_pu.repeat(2)

        # Add zeros to indicate it is a line, and pad with nan to account for
        # missing phase shift.
        e = edge_index.shape[1] # b/c coo matrix
        edge_features = np.vstack([np.zeros(e),         # trafo?
                                   r_pu,                # r_pu
                                   x_pu,                # x_pu
                                   np.nan*np.ones(e)    # phase_shift
                                   ]).T

        return edge_index, edge_features

    def get_trafo_features(net):
        # TODO: Add charging susceptance - b (p.u.), transformer tap ratio - tau

        # Similar to get_line_features.
        edge_index = net.trafo.loc[:, ['hv_bus', 'lv_bus',
                                       'lv_bus', 'hv_bus']].values
        edge_index = edge_index.reshape(-1, 2).T

        # Impedance calculated as shown in pandapower docs:
        # https://pandapower.readthedocs.io/en/v2.2.1/elements/trafo.html#impedance-values
        # where vk_percent is short-circuit voltage and vkr_percent is the real
        # part of short-circuit voltage (%).
        z_pu = (net.trafo['vk_percent'].values / 100)*(1000 / net.trafo['sn_mva'].values)
        r_pu = (net.trafo['vkr_percent'].values / 100)*(1000 / net.trafo['sn_mva'].values)
        x_pu = np.sqrt(np.square(z_pu) - np.square(r_pu))

        # Add phase shift angle (deg) as additional feature.
        phase_shift = net.trafo['vk_percent'].values
        
        # Repeat the features (to match edge_index) and create feature matrix.
        e = edge_index.shape[1] # b/c coo matrix
        edge_features = np.vstack([np.ones(e),              # trafo?
                                   r_pu.repeat(2),          # r_pu
                                   x_pu.repeat(2),          # x_pu
                                   phase_shift.repeat(2)    # phase_shift
                                   ]).T

        return edge_index, edge_features
    
    A_line, E_line = get_line_features(net)
    A_trafo, E_trafo = get_trafo_features(net)
    
    # Combine and return the line and trafo features.
    A = np.hstack([A_line, A_trafo])
    E = np.vstack([E_line, E_trafo])
    return A, E

### Create datasets for GNN-based model development using the generated grids

In [10]:
dataset_splits = ['train', 'val', 'test']

for split in dataset_splits:
    # Gather all the grids for a particular dataset split
    generated_grid_files = []
    for dir in generated_grid_base_dirs:
        generated_grid_dir = os.path.join(dir, split)
        generated_grids = os.listdir(generated_grid_dir)
        generated_grid_files.extend([os.path.join(generated_grid_dir, f) for f in generated_grids])

    X = []
    Y = []
    A = [] # In COO matrix format, aka edge_index
    E = []

    for f in generated_grid_files:
        net = pp.from_json(f)

        X_i, Y_i = get_node_features(net)
        A_i, E_i = get_edge_features(net)
        
        X.append(X_i)
        Y.append(Y_i)
        A.append(A_i)
        E.append(E_i)

    np.save(os.path.join(output_dir, f'X_{split}.npy'), X)
    np.save(os.path.join(output_dir, f'Y_{split}.npy'), Y)
    np.save(os.path.join(output_dir, f'A_{split}.npy'), A)
    np.save(os.path.join(output_dir, f'E_{split}.npy'), E) 